In [1]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import pandas as pd
import os
import pint
import json
import pprint

from pygama.flow import DataLoader
from pygama.flow import FileDB
from lgdo.lh5_store import LH5Store
from lgdo import ls, Table, WaveformTable
from dspeed import build_dsp

# Setup FileDB and DataLoader

In [2]:
# Run this to perform a scan of all files and create a FileDB from scratch, then save it to disk
fdb = FileDB(config="metadata/dataloader_configs/cage_filedb_config.json")
fdb.scan_tables_columns()
fdb.to_disk("cage_filedb.lh5", "o")

cannot find 'ORSIS3302DecoderForEnergy/raw' in /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage_run385_cyc2527_raw.lh5
cannot find 'ORSIS3302DecoderForEnergy/raw' in /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage_run385_cyc2528_raw.lh5
cannot find 'ORSIS3302DecoderForEnergy/raw' in /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage_run386_cyc2529_raw.lh5
cannot find 'ORSIS3302DecoderForEnergy/raw' in /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage_run386_cyc2530_raw.lh5
cannot find 'ORSIS3302DecoderForEnergy/raw' in /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage_run386_cyc2531_raw.lh5
cannot find 'ORSIS3302DecoderForEnergy/raw' in /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage_run386_cyc2532_raw.lh5
cannot find 'ORSIS3302DecoderForEnergy/raw' in /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage_run387_cyc2533_raw.lh5
cannot find 'ORSIS3302DecoderForEnergy/raw' in /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage_run387_cyc2534_raw.lh5
cannot find 'ORSIS3302DecoderForEnergy/raw' in /global/cfs/cdirs

In [ ]:
# Run this to load an existing FileDB from disk
fdb = FileDB("cage_filedb.lh5")

In [3]:
dl = DataLoader(config="metadata/dataloader_configs/cage_loader_config.json",
                filedb=fdb)

In [ ]:
cyc = 3207
dl.reset()
dl.set_files(f"cycle == {cyc}")
dl.set_output(fmt="pd.DataFrame", columns=["energy"])
el = dl.build_entry_list(save_output_columns=True)
data = dl.load(el)

Below you can check the spectrum to make sure it looks okay, then you can zoom in and pick out where the 1460 kev peak is. 

In [ ]:
%matplotlib widget

In [ ]:
%matplotlib inline

In [ ]:
plt.figure()
#plt.yscale('log')
plt.hist(data['energy'], bins = np.linspace(0, 4e6, 1000))
plt.xlabel('energy')
plt.ylabel('count')

In [ ]:
plt.figure()
plt.hist(data['energy'], bins = np.linspace(1.75e6, 1.85e6, 40))
plt.xlabel('energy')
plt.ylabel('count')

In [ ]:
# input the low and high ends of the 1460 kev peak here
elo = 1.77e6
ehi = 1.80e6

# Load 1460 keV Waveforms

In [ ]:
dl.reset()
dl.set_files(f"cycle == {cyc}")
dl.set_cuts({"hit": f"energy > {elo} and energy < {ehi}"})
dl.set_output(columns=["waveform"])
wfs = dl.load()

In [ ]:
wfs['waveform']['dt'].attrs

In [ ]:
# Save waveforms to a temporary "raw" file
raw_file = "1460raw_temp.lh5"
# sto.write_object(obj=wfs["energy"], name="energy", lh5_file=raw_file, group="raw")
sto = LH5Store()
sto.write_object(obj=wfs, name="40K", lh5_file=raw_file, wo_mode="of")

In [ ]:
ureg = pint.UnitRegistry()
per_unit = 1/(wfs['waveform']['dt'].nda[0] * ureg(wfs['waveform']['dt'].attrs['units']).units)
per_us = per_unit.to("1/us")
print(per_unit)
print(per_us)

# First Pass DSP
We run the DSP once to find a reasonable guess for `pz_tau` and bins for our energy histogram

In [ ]:
dsp_db = {
    "40K": {
        "etrap": {
            "rise": "8*us",
            "flat": "2*us"
        },
        "pz": {
            "tau": "70*us"
        },
        "dcr_trap": {
            "rise": "8*us",
            "flat": "2*us"
        },
        "ctc": {
            "alpha": 1
        }
    }
}
dsp_config = {
  "outputs": [
    "trapEmax", "lt_slope"
  ],
  "processors":{
    "bl, bl_sig, bl_slope, bl_intercept":{
      "function": "linear_slope_fit",
      "module": "pygama.dsp.processors",
      "args" : ["waveform[0: 3500]", "bl","bl_sig", "bl_slope","bl_intercept"],
      "unit": ["ADC","ADC","ADC","ADC"]
    },
    "wf_blsub":{
      "function": "subtract",
      "module": "numpy",
      "args": ["waveform", "bl", "wf_blsub"],
      "prereqs": ["waveform", "bl"],
      "unit": "ADC"
    },
    "wf_logtail": {
      "function": "log",
      "module": "numpy",
      "args": ["wf_blsub[4250:8000]", "wf_logtail"],
      "unit": "ADC",
      "prereqs": ["wf_blsub"]
    },
    "lt_mean, lt_sig, lt_slope, lt_int": {
        "function": "linear_slope_fit",
        "module": "pygama.dsp.processors",
        "args": ["wf_logtail", "lt_mean", "lt_sig", "lt_slope", "lt_int"],
        "unit": ["ADC", "ADC", "ADC", "ADC"],
        "prereqs": ["wf_logtail"]
    },
    "wf_pz": {
      "function": "pole_zero",
      "module": "pygama.dsp.processors",
      "args": ["wf_blsub", "db.pz.tau", "wf_pz"],
      "prereqs": ["wf_blsub"],
      "unit": "ADC",
    },
    "wf_etrap": {
      "function": "trap_norm",
      "module": "pygama.dsp.processors",
      "prereqs": ["wf_pz"],
      "args": ["wf_pz", "db.etrap.rise", "db.etrap.flat", "wf_etrap"],
      "unit": "ADC"
    },
    "trapEmax": {
      "function": "amax",
      "module": "numpy",
      "args": ["wf_etrap", 1, "trapEmax"],
      "kwargs": {"signature":"(n),()->()", "types":["fi->f"]},
      "unit": "ADC",
      "prereqs": ["wf_etrap"]
    },
    "trapEmax_ctc": {
      "function": "add",
      "module": "numpy",
      "args": ["trapEmax", "db.ctc.alpha*dcr", "trapEmax_ctc"],
      "unit": "ADC",
      "prereqs": ["trapEmax", "dcr"]
    },
    "tp_min, tp_max, wf_min, wf_max":{
      "function": "min_max",
      "module": "pygama.dsp.processors",
      "args": ["wf_blsub", "tp_min", "tp_max", "wf_min", "wf_max"],
      "unit": ["ns","ns","ADC", "ADC"],
      "prereqs": ["wf_blsub"]
    },
    "pz_mean, pz_sig, pz_slope, pz_int": {
        "function": "linear_slope_fit",
        "module": "pygama.dsp.processors",
        "args": ["wf_pz[5750:8000]", "pz_mean", "pz_sig", "pz_slope", "pz_int"],
        "unit": ["ADC", "ADC", "ADC", "ADC"],
        "prereqs": ["wf_pz"]
    },
    "wf_dcr_trap": {
        "function": "trap_norm",
        "module": "pygama.dsp.processors",
        "args": ["wf_pz", "db.dcr_trap.rise", "db.dcr_trap.flat", "wf_dcr_trap"],
        "defaults" : {"db.dcr_trap.rise":"7*us", "db.dcr_trap.flat":"20*us"},
        "unit": "ADC",
        "prereqs": ["wf_pz"]
    },
    "dcr": {
        "function": "fixed_time_pickoff",
        "module": "pygama.dsp.processors",
        "args": ["wf_dcr_trap", "db.dcr.ftp", "i", "dcr"],
        "defaults" : {"db.dcr.ftp" : "80*us"},
        "unit": "ADC",
        "prereqs": ["wf_dcr_trap"]
    }
  }
}


In [ ]:
dsp_file = "1460dsp_temp.lh5"
build_dsp(f_raw=raw_file, f_dsp=dsp_file, dsp_config=dsp_config, database=dsp_db, write_mode='r', n_max=100)

In [ ]:
pk_table, _ = sto.read_object("40K", dsp_file)
pk_df = pk_table.get_dataframe()

In [ ]:
lt_tau = 1 / (pk_df['lt_slope'].mean()*per_us.magnitude) # ADC/sample * samples/us = ADC/us
tau_range = -np.arange(lt_tau - 1, lt_tau + 1, .2)

In [ ]:
emed = pk_df.median()['trapEmax']
ebins = np.linspace(emed - 0.02*emed, emed + 0.02*emed, 20)

In [ ]:
rise_range = np.arange(5, 15)
dcrrise_range = np.arange(2, 5)
dcrflat_range = np.arange(18, 21)
alpha_range = np.linspace(0.5, 15, 10)

In [ ]:
#plt.hist(pk_df['trapEmax_ctc'], bins=ebins, histtype='step')
plt.hist(pk_df['trapEmax'], bins=ebins, histtype='step')

In [ ]:
print(lt_tau)
print(tau_range)
print(rise_range)
print(dcrrise_range)
print(dcrflat_range)
print(ebins)
print(alpha_range)

# DSP Loop

In [ ]:
dsp_config["outputs"] = ["pz_slope", "trapEmax", "dcr", "trapEmax_ctc"]
dsp_config["processors"].pop("wf_logtail")
dsp_config["processors"].pop("lt_mean, lt_sig, lt_slope, lt_int")

## Pole Zero Loop

Here we are finding the optimum time constant for the decay of the tail, determined by which one gives the flattest slope of the tail once pole-zero corrected

In [ ]:
results = None
for tau in tau_range:
    dsp_db["40K"]["pz"]["tau"] = str(tau) + " * us"
        
    dsp_file = "1460dsp_temp.lh5"
    build_dsp(f_raw=raw_file, f_dsp=dsp_file, dsp_config=dsp_config, database=dsp_db, write_mode='r', n_max=20)
    
    pk_table, _ = sto.read_object("40K", dsp_file)
    res = pd.DataFrame({
        "tau": [tau],
        "pz_slope_avg": [np.nanmean(np.abs(pk_table['pz_slope'].nda))]
    })
    if results is None:
        results = res
    else:
        results = pd.concat([results, res], ignore_index=True)
print(results)

If the best result is at the edge of the range, reset `tau_range` to explore more values, and run the above loop again

In [ ]:
tau_range = np.linspace(65.2, 65.8, 10)

Set the DB value to the best tau we found and then plot `wf_pz` to look at the tails

In [ ]:
tau = results.abs().sort_values("pz_slope_avg").iloc[0]['tau']
print(tau)
dsp_db["40K"]["pz"]["tau"] = str(tau) + "* us"

In [ ]:
dsp_config["outputs"].append("wf_pz")
build_dsp(f_raw=raw_file, f_dsp=dsp_file, dsp_config=dsp_config, database=dsp_db, write_mode='r')
pk_table, _ = sto.read_object("40K", dsp_file)
plt.figure()
for wf in pk_table['wf_pz']['values'].nda:
    plt.plot(wf[4250:])
dsp_config["outputs"] = dsp_config["outputs"][:-1]

## Energy Trap Loop

We vary the `rise` parameter in the energy trapezoid to find which one gives the highest peak for the 1460 keV line. 

In [ ]:
results = None
for rise in rise_range:
    dsp_db["40K"]["etrap"]["rise"] = str(rise) + " * us"
        
    dsp_file = "1460dsp_temp.lh5"
    build_dsp(f_raw=raw_file, f_dsp=dsp_file, dsp_config=dsp_config, database=dsp_db, write_mode='r')
    
    pk_table, _ = sto.read_object("40K", dsp_file)
    
    # These lines will plot each energy histogram
    #plt.figure()
    #plt.yscale('log')
    #ehist, _, _ = plt.hist(pk_table['trapEmax'].nda, bins = np.linspace(3300, 3600, 50), label=rise)
    #plt.legend()
    
    # This one just runs and tells the answer
    ehist, _ = np.histogram(pk_table['trapEmax'].nda, bins = ebins)
    
    res = pd.DataFrame({
        "rise": [rise],
        "peak_height": [np.max(ehist)]
    })
    if results is None:
        results = res
    else:
        results = pd.concat([results, res], ignore_index=True)
print(results)

In [ ]:
rise_range = np.arange(1, 18)

In [ ]:
plt.figure()
plt.plot(results["peak_height"])
plt.xlabel("etrap rise (us)")
plt.ylabel("peak height")

In [ ]:
rise = results.abs().sort_values("peak_height").iloc[-1]['rise']
print(rise)
dsp_db["40K"]["etrap"]["rise"] = str(rise) + "* us"

## DCR Loop

We do a grid-search on both the `rise` and `flat` parameters for the DCR trapezoid that give us a DCR centered around zero.

In [ ]:
results = None
for rise in dcrrise_range:
    for flat in dcrflat_range:
        dsp_db["40K"]["dcr_trap"]["rise"] = str(rise) + " * us"
        dsp_db["40K"]["dcr_trap"]["flat"] = str(flat) + " * us"

        dsp_file = "1460dsp_temp.lh5"
        build_dsp(f_raw=raw_file, f_dsp=dsp_file, dsp_config=dsp_config, database=dsp_db, write_mode='r')

        pk_table, _ = sto.read_object("40K", dsp_file)

        res = pd.DataFrame({
            "rise": [rise],
            "flat": [flat],
            "dcr_mean": [np.abs(np.mean(pk_table['dcr'].nda))]
        })
        if results is None:
            results = res
        else:
            results = pd.concat([results, res], ignore_index=True)
print(results)

If the best result is at the edge of the range, reset `dcrrise_range` and `dcrflat_range` to explore more values, and run the above loop again

In [ ]:
dcrrise_range = np.linspace(3.75, 4.5, 4)
dcrflat_range = np.linspace(19.75, 20.5, 4)

Set the DB value to the best tau we found and then plot `wf_pz` to look at the tails

In [ ]:
dcrrise = results.abs().sort_values("dcr_mean").iloc[0]['rise']
dcrflat = results.abs().sort_values("dcr_mean").iloc[0]['flat']
print(dcrrise, dcrflat)
dsp_db["40K"]["dcr_trap"]["rise"] = str(dcrrise) + "* us"
dsp_db["40K"]["dcr_trap"]["flat"] = str(dcrflat) + "* us"

## Charge Trapping Correction

We want to correct our energy estimation by adding a value to `trapEftp` based on the DCR. 
- trapEftp_ctc = trapEftp + alpha*DCR

We are trying to optimize a value for `alpha` that gives the highest peak for the 1460 keV line

In [ ]:
results = None
for alpha in alpha_range:
    dsp_db["40K"]["ctc"]["alpha"] = str(alpha)
        
    dsp_file = "1460dsp_temp.lh5"
    build_dsp(f_raw=raw_file, f_dsp=dsp_file, dsp_config=dsp_config, database=dsp_db, write_mode='r')
    
    pk_table, _ = sto.read_object("40K", dsp_file)
    
    # These lines will plot each energy histogram
    #plt.figure()
    #plt.yscale('log')
    #ehist, _, _ = plt.hist(pk_table['trapEmax_ctc'].nda, bins = ebins, label=alpha)
    #plt.legend()
    
    # This one just runs and tells the answer
    ehist, _ = np.histogram(pk_table['trapEmax_ctc'].nda, bins = ebins)
    
    res = pd.DataFrame({
        "alpha": [alpha],
        "peak_height": [np.max(ehist)]
    })
    if results is None:
        results = res
    else:
        results = pd.concat([results, res], ignore_index=True)
print(results)

If the best result is at the edge of the range, reset `alpha_range` to explore more values, and run the above loop again

In [ ]:
alpha_range = np.append([0], np.linspace(0, 0.9, 10))

In [ ]:
alpha = results.abs().sort_values("peak_height").iloc[-1]['alpha']
print(alpha)
dsp_db["40K"]["ctc"]["alpha"] = alpha

In [ ]:
build_dsp(f_raw=raw_file, f_dsp=dsp_file, dsp_config=dsp_config, database=dsp_db, write_mode='r')
pk_table, _ = sto.read_object("40K", dsp_file)
plt.figure()
plt.hist(pk_table['trapEmax'].nda, bins=ebins, histtype='step')
plt.hist(pk_table['trapEmax_ctc'].nda, bins=ebins, histtype='step')

In [ ]:
plt.figure()
plt.hist2d(pk_table['trapEmax'].nda, pk_table['dcr'].nda, bins=(ebins, np.arange(-20, 20)))
plt.colorbar()

In [ ]:
plt.figure()
plt.hist2d(pk_table['trapEmax_ctc'].nda, pk_table['dcr'].nda, bins=(ebins, np.arange(-20, 20)))
plt.colorbar()

# Run On One File

In [ ]:
raw = fdb.df.query(f"cycle == {cyc}").iloc[0]
raw = os.path.join(fdb.data_dir, fdb.tier_dirs['raw'], raw['raw_file'])
dsp = f"cycle{cyc}_testdsp.lh5"

In [ ]:
# Write results to dsp_07.json defaults
with open('./metadata/dsp/dsp_07.json') as f:
    test_config = json.load(f)
test_config['processors']['wf_pz']['defaults'] = {"db.pz.tau": dsp_db['40K']['pz']['tau']}
test_config['processors']['wf_etrap']['defaults'] = {"db.etrap.rise": dsp_db['40K']['etrap']['rise'], 
                                                  "db.etrap.flat": dsp_db['40K']['etrap']['flat']}
test_config['processors']['wf_dcr_trap']['defaults'] = {"db.dcr_trap.rise": dsp_db['40K']['dcr_trap']['rise'], 
                                                  "db.dcr_trap.flat": dsp_db['40K']['dcr_trap']['flat']}
test_config['processors']['trapEmax_ctc']['defaults'] = {"db.ctc.alpha": dsp_db['40K']['ctc']['alpha']}

In [ ]:
pprint.pprint(dsp_db)

In [ ]:
pprint.pprint(test_config)

In [ ]:
build_dsp(f_raw=raw, f_dsp=dsp, dsp_config=test_config, write_mode='r')

In [ ]:
ls(dsp, 'ORSIS3302DecoderForEnergy/dsp/*')

In [ ]:
dsp_table, _ = sto.read_object("ORSIS3302DecoderForEnergy/dsp", dsp)

In [ ]:
# energy histogram
plt.figure()
plt.yscale('log')
plt.hist(dsp_table['trapEmax'].nda, bins=np.linspace(3140, 3170, 500))

In [ ]:
# check linearity
k40_peak = 3155
plt.figure()
plt.yscale('log')
plt.hist(dsp_table['trapEmax'].nda*(1460/k40_peak), bins=np.linspace(2600, 2630, 20))

In [ ]:
# dcr histogram
plt.figure()
plt.hist(dsp_table['dcr'].nda, bins=np.arange(-50, 50))

In [ ]:
plt.figure()
plt.hist2d(dsp_table['trapEmax'].nda, dsp_table['dcr'].nda, 
           bins = (np.linspace(0, 10000, 100), np.arange(-200, 200)), 
           norm=colors.LogNorm())
plt.xlabel("trapEmax")
plt.ylabel("DCR")
plt.axhline(0, color='r')
plt.xlim(3000, 7000)
plt.ylim(-50, 50)

In [ ]:
plt.figure()
plt.hist2d(dsp_table['trapEmax_ctc'].nda, dsp_table['dcr'].nda, 
           bins = (np.linspace(0, 10000, 100), np.arange(-200, 200)), 
           norm=colors.LogNorm())
plt.xlabel("trapEmax_ctc")
plt.ylabel("DCR")
plt.axhline(0, color='r')
plt.xlim(3000, 7000)
plt.ylim(-50, 50)

In [ ]:
plt.figure()
plt.hist2d(dsp_table['trapEmax_ctc'].nda, dsp_table['dcr'].nda, 
           bins = (np.linspace(0, 10000, 100), np.arange(-200, 200)), 
           norm=colors.LogNorm())
plt.xlabel("trapEmax_ctc")
plt.ylabel("DCR")
plt.axhline(0, color='r')

In [ ]:
plt.figure()
plt.hist2d(dsp_table['trapEmax'].nda, dsp_table['dcr'].nda, 
           bins = (np.linspace(0, 10000, 100), np.arange(-200, 200)), 
           norm=colors.LogNorm())
plt.xlabel("trapEmax")
plt.ylabel("DCR")
plt.axhline(0, color='r')

In [ ]:
# energy histogram
plt.figure()
plt.yscale('log')
plt.hist(dsp_table['trapEmax'].nda, bins=np.linspace(3000, 3500, 50), histtype='step')
plt.hist(dsp_table['trapEmax_ctc'].nda, bins=np.linspace(3000, 3500, 50), histtype='step')

In [ ]:
# energy histogram
plt.figure()
plt.yscale('log')
plt.hist(dsp_table['trapEmax'].nda, bins=np.linspace(5750, 6250, 50), histtype='step')
plt.hist(dsp_table['trapEmax_ctc'].nda, bins=np.linspace(5750, 6250, 50), histtype='step')

In [ ]:
test_config

In [ ]:
# Write this configuration to a file
with open(f'./metadata/dsp/dsp_cyc{cyc}.json', 'w') as f:
    json.dump(test_config, f)